In [14]:
import pandas as pd
import numpy as np
import pyfixest as pf
import scipy
from scipy import stats
import altair as alt
import statsmodels.formula.api as smf

In [15]:
# Read state policy data
df = pd.read_csv('data/cspp_data_2025-11-20.csv')
df.columns
df

,year,st,stateno,state,state_fips,state_icpsr,avgec_low,avgec_mid,avgec_high,avgsoc_low,...,sqli,health_rank,infconsent,w_ec_access,solaw,vaaban,frps,drugs_medical_marijuana,gambling_lottery_adoption,guncontrol_assaultweapon_ban
0,1960,AK,2.0,Alaska,2,81,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1,1961,AK,2.0,Alaska,2,81,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2,1962,AK,2.0,Alaska,2,81,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3,1963,AK,2.0,Alaska,2,81,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4,1964,AK,2.0,Alaska,2,81,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3106,2016,WY,50.0,Wyoming,56,68,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
3107,2017,WY,50.0,Wyoming,56,68,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3108,2018,WY,50.0,Wyoming,56,68,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3109,2019,WY,50.0,Wyoming,56,68,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Read NEA teacher salary data
salaries_df_wide = pd.read_csv('data/teacher_salaries.csv')
salaries_df = pd.melt(salaries_df_wide, id_vars = ['State'], var_name = 'Year', value_name = 'Salary')
salaries_df.rename(columns = {'State': 'state', 'Year': 'year', 'Salary': 'salary'}, inplace = True)

salaries_df['year'] = salaries_df['year'].astype(int)
salaries_df['salary'] = salaries_df['salary'].replace({',': ''}, regex=True).astype(int)

salaries_df

,state,year,salary
0,United States,1990,31367
1,Alabama,1990,24828
2,Alaska,1990,43153
3,Arizona,1990,29402
4,Arkansas,1990,22352
...,...,...,...
1607,Virginia,2020,57665
1608,Washington,2020,76743
1609,West Virginia,2020,50238
1610,Wisconsin,2020,59431


In [17]:
df_filtered = df[['year', 'st', 'state', 'agencyfeesprohibited']]

# incl_states = ['Wisconsin', 'Minnesota', 'Michigan']

# Filter for states and years 
# df_filtered = df_filtered[df_filtered['state'].isin(incl_states)]
df_filtered = df_filtered[(df_filtered['year'] >= 1990) & (df_filtered['year'] <= 2018)]

df_filtered

,year,st,state,agencyfeesprohibited
30,1990,AK,Alaska,0.0
31,1991,AK,Alaska,0.0
32,1992,AK,Alaska,0.0
33,1993,AK,Alaska,0.0
34,1994,AK,Alaska,0.0
...,...,...,...,...
3104,2014,WY,Wyoming,1.0
3105,2015,WY,Wyoming,1.0
3106,2016,WY,Wyoming,1.0
3107,2017,WY,Wyoming,1.0


In [25]:
df_merged = pd.merge(df_filtered, salaries_df, on = ['state', 'year'], how = 'left')

# Drop District of Columbia
df_merged = df_merged[df_merged['state'] != 'District of Columbia']

# Adjust variable types
df_merged['agencyfeesprohibited'] = df_merged['agencyfeesprohibited'].astype(int)
df_merged["state"] = df_merged["state"].astype("string")
df_merged["st"] = df_merged["st"].astype("string")

# Identify states that ever had a 1 under 'agencyfeesprohibited' i.e., ever had a RTW law pass
states_with_fees_prohibited = df_merged[df_merged['agencyfeesprohibited'] == 1]['st'].unique()

# Exclude states that ever had RTW except for Wisconsin (our sole treatment)
df = df_merged[~((df_merged['st'].isin(states_with_fees_prohibited)) & (df_merged['st'] != 'WI'))]

df

,year,st,state,agencyfeesprohibited,salary
0,1990,AK,Alaska,0,43153
1,1991,AK,Alaska,0,43427
2,1992,AK,Alaska,0,44661
3,1993,AK,Alaska,0,46701
4,1994,AK,Alaska,0,47512
...,...,...,...,...,...
1416,2014,WI,Wisconsin,1,53679
1417,2015,WI,Wisconsin,1,52264
1418,2016,WI,Wisconsin,1,54115
1419,2017,WI,Wisconsin,1,51439
